### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Trabajar con archivos JSON

In [ ]:
dbutils.fs.mkdirs("/FileStore/tables/json/")

Out[52]: True

#### Leer un archivo JSON

##### Ejemplo 1

In [ ]:
json_df = spark.read.option('inferschema','true').json('/FileStore/tables/json/constructor.json')
json_df.printSchema()
json_df.show(5, truncate=False)

root
 |-- constructorId: long (nullable = true)
 |-- constructorRef: string (nullable = true)
 |-- name: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- url: string (nullable = true)

+-------------+--------------+----------+-----------+------------------------------------------------------------+
|constructorId|constructorRef|name      |nationality|url                                                         |
+-------------+--------------+----------+-----------+------------------------------------------------------------+
|1            |mclaren       |McLaren   |British    |http://en.wikipedia.org/wiki/McLaren                        |
|2            |bmw_sauber    |BMW Sauber|German     |http://en.wikipedia.org/wiki/BMW_Sauber                     |
|3            |williams      |Williams  |British    |http://en.wikipedia.org/wiki/Williams_Grand_Prix_Engineering|
|4            |renault       |Renault   |French     |http://en.wikipedia.org/wiki/Renault_in_Formula_

Definiendo un schema obtenemos el mismo resultado anterior

In [ ]:
constructor_schema = 'constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING'
json_df = spark.read.schema(constructor_schema).json('/FileStore/tables/json/constructor.json')
json_df.printSchema()
json_df.show(5, truncate=False)

root
 |-- constructorId: integer (nullable = true)
 |-- constructorRef: string (nullable = true)
 |-- name: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- url: string (nullable = true)

+-------------+--------------+----------+-----------+------------------------------------------------------------+
|constructorId|constructorRef|name      |nationality|url                                                         |
+-------------+--------------+----------+-----------+------------------------------------------------------------+
|1            |mclaren       |McLaren   |British    |http://en.wikipedia.org/wiki/McLaren                        |
|2            |bmw_sauber    |BMW Sauber|German     |http://en.wikipedia.org/wiki/BMW_Sauber                     |
|3            |williams      |Williams  |British    |http://en.wikipedia.org/wiki/Williams_Grand_Prix_Engineering|
|4            |renault       |Renault   |French     |http://en.wikipedia.org/wiki/Renault_in_Formu

##### Ejemplo 2

In [ ]:
json_df = spark.read.option('inferschema','true').json('/FileStore/tables/json/drivers.json')
json_df.printSchema()
json_df.show(5, truncate=False)

root
 |-- code: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- driverId: long (nullable = true)
 |-- driverRef: string (nullable = true)
 |-- name: struct (nullable = true)
 |    |-- forename: string (nullable = true)
 |    |-- surname: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- number: string (nullable = true)
 |-- url: string (nullable = true)

+----+----------+--------+----------+--------------------+-----------+------+----------------------------------------------+
|code|dob       |driverId|driverRef |name                |nationality|number|url                                           |
+----+----------+--------+----------+--------------------+-----------+------+----------------------------------------------+
|HAM |1985-01-07|1       |hamilton  |{Lewis, Hamilton}   |British    |44    |http://en.wikipedia.org/wiki/Lewis_Hamilton   |
|HEI |1977-05-10|2       |heidfeld  |{Nick, Heidfeld}    |German     |\N    |http://en.wikipedia.org/wi

In [ ]:
# Vamos a crear el schema para leer este archivo json
from pyspark.sql.types import *
from pyspark.sql.functions import *

name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True),
                                 ])

drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                    StructField("driverRef", StringType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("code", StringType(), True),
                                    StructField("name", name_schema),
                                    StructField("dob", DateType(), True),
                                    StructField("nationality", StringType(), True),
                                    StructField("url", StringType(), True)
                                   ])

json_df = spark.read.format('json').schema(drivers_schema).json('/FileStore/tables/json/drivers.json')
json_df.printSchema()
json_df.show(5, truncate=False)

root
 |-- driverId: integer (nullable = true)
 |-- driverRef: string (nullable = true)
 |-- number: integer (nullable = true)
 |-- code: string (nullable = true)
 |-- name: struct (nullable = true)
 |    |-- forename: string (nullable = true)
 |    |-- surname: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- nationality: string (nullable = true)
 |-- url: string (nullable = true)

+--------+----------+------+----+--------------------+----------+-----------+----------------------------------------------+
|driverId|driverRef |number|code|name                |dob       |nationality|url                                           |
+--------+----------+------+----+--------------------+----------+-----------+----------------------------------------------+
|1       |hamilton  |44    |HAM |{Lewis, Hamilton}   |1985-01-07|British    |http://en.wikipedia.org/wiki/Lewis_Hamilton   |
|2       |heidfeld  |null  |HEI |{Nick, Heidfeld}    |1977-05-10|German     |http://en.wikipedia.org/

In [ ]:
json_df_modif = json_df.withColumnRenamed('driverId','driver_Id'). \
                        withColumnRenamed('driverRef','driver_Ref'). \
                        withColumn('name',concat(col('name.forename'), lit(' '), col('name.surname')))

json_df_modif.printSchema()
json_df_modif.show(5, truncate=False)

root
 |-- driver_Id: integer (nullable = true)
 |-- driver_Ref: string (nullable = true)
 |-- number: integer (nullable = true)
 |-- code: string (nullable = true)
 |-- name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- nationality: string (nullable = true)
 |-- url: string (nullable = true)

+---------+----------+------+----+-----------------+----------+-----------+----------------------------------------------+
|driver_Id|driver_Ref|number|code|name             |dob       |nationality|url                                           |
+---------+----------+------+----+-----------------+----------+-----------+----------------------------------------------+
|1        |hamilton  |44    |HAM |Lewis Hamilton   |1985-01-07|British    |http://en.wikipedia.org/wiki/Lewis_Hamilton   |
|2        |heidfeld  |null  |HEI |Nick Heidfeld    |1977-05-10|German     |http://en.wikipedia.org/wiki/Nick_Heidfeld    |
|3        |rosberg   |6     |ROS |Nico Rosberg     |1985-06-27|German    

##### Ejemplo 3

In [ ]:
json_df = spark.read.option('inferschema','true').option('multiline',True).json('/FileStore/tables/json/pit_stops.json')
json_df.printSchema()
json_df.show(5)

root
 |-- driverId: long (nullable = true)
 |-- duration: string (nullable = true)
 |-- lap: long (nullable = true)
 |-- milliseconds: long (nullable = true)
 |-- raceId: long (nullable = true)
 |-- stop: long (nullable = true)
 |-- time: string (nullable = true)

+--------+--------+---+------------+------+----+--------+
|driverId|duration|lap|milliseconds|raceId|stop|    time|
+--------+--------+---+------------+------+----+--------+
|     153|  26.898|  1|       26898|   841|   1|17:05:23|
|      30|  25.021|  1|       25021|   841|   1|17:05:52|
|      17|  23.426| 11|       23426|   841|   1|17:20:48|
|       4|  23.251| 12|       23251|   841|   1|17:22:34|
|      13|  23.842| 13|       23842|   841|   1|17:24:10|
+--------+--------+---+------------+------+----+--------+
only showing top 5 rows



In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

pit_schema = StructType(fields=[StructField("raceId", IntegerType(), True),
                                StructField("driverId", IntegerType(), True),
                                StructField("stop", StringType(), True),
                                StructField("lap", IntegerType(), True),
                                StructField("time", StringType(), True),
                                StructField("duration", StringType(), True),
                                StructField("miliseconds", StringType(), True)
                                ])

json_df = spark.read.format('json').schema(pit_schema).option('multiline',True).json('/FileStore/tables/json/pit_stops.json')
json_df.printSchema()
json_df.show(5)

root
 |-- raceId: integer (nullable = true)
 |-- driverId: integer (nullable = true)
 |-- stop: string (nullable = true)
 |-- lap: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- miliseconds: string (nullable = true)

+------+--------+----+---+--------+--------+-----------+
|raceId|driverId|stop|lap|    time|duration|miliseconds|
+------+--------+----+---+--------+--------+-----------+
|   841|     153|   1|  1|17:05:23|  26.898|       null|
|   841|      30|   1|  1|17:05:52|  25.021|       null|
|   841|      17|   1| 11|17:20:48|  23.426|       null|
|   841|       4|   1| 12|17:22:34|  23.251|       null|
|   841|      13|   1| 13|17:24:10|  23.842|       null|
+------+--------+----+---+--------+--------+-----------+
only showing top 5 rows



##### Ejemplo 4

In [ ]:
from pyspark.sql.functions import *

json_df = spark.read.option('multiline',True).json('/FileStore/tables/json/perros.json')
json_df.printSchema()
display(json_df)

root
 |-- persons: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- age: long (nullable = true)
 |    |    |-- dogs: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- name: string (nullable = true)



persons
"List(List(30, List(Fido, Fluffy), Keith), List(46, List(Spot), Donna))"


In [ ]:
json_df_explode = json_df.select(explode(json_df.persons).alias('persons'))
json_df_explode.printSchema()
json_df_explode.display()

root
 |-- persons: struct (nullable = true)
 |    |-- age: long (nullable = true)
 |    |-- dogs: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- name: string (nullable = true)



persons
"List(30, List(Fido, Fluffy), Keith)"
"List(46, List(Spot), Donna)"


In [ ]:
owners = json_df_explode.select(col('persons.name').alias('owner'), col('persons.age').alias('age'), explode('persons.dogs').alias('dog'))

owners.printSchema()
owners.display()

root
 |-- owner: string (nullable = true)
 |-- age: long (nullable = true)
 |-- dog: string (nullable = true)



owner,age,dog
Keith,30,Fido
Keith,30,Fluffy
Donna,46,Spot


##### Ejemplo 5

In [ ]:
json_df = spark.read.option('inferschema','true').json('/FileStore/tables/json/accounting.json')
json_df.printSchema()
display(json_df)

root
 |-- accounting: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- age: long (nullable = true)
 |    |    |-- first_name: string (nullable = true)
 |    |    |-- last_name: string (nullable = true)



accounting
"List(List(23, John, Doe), List(32, Mary, Smith))"


In [ ]:
from pyspark.sql.functions import *

json_df = spark.read.option('multiline',True).json('/FileStore/tables/json/accounting.json')
json_df.printSchema()
display(json_df)

root
 |-- accounting: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- age: long (nullable = true)
 |    |    |-- first_name: string (nullable = true)
 |    |    |-- last_name: string (nullable = true)



accounting
"List(List(23, John, Doe), List(32, Mary, Smith))"


In [ ]:
json_df_modif = json_df.select(explode(json_df.accounting).alias('account')). \
                        select('account.age','account.first_name','account.last_name')

display(json_df_modif)

age,first_name,last_name
23,John,Doe
32,Mary,Smith


##### Ejemplo 6
Estamos trabajando con tipos de datos 'struct'. La funcion 'explode' solo se puede utilizar en 'arrays' y 'maps'.

In [ ]:
from pyspark.sql.functions import *
df = spark.read.option('multiline',True).json('/FileStore/tables/json/transacciones_complejas.json')
df.printSchema()
display(df)

root
 |-- EMPRESA: struct (nullable = true)
 |    |-- ID_EMPRESA: long (nullable = true)
 |    |-- NOMBRE: string (nullable = true)
 |-- FECHA: string (nullable = true)
 |-- MONTO: long (nullable = true)
 |-- PERSONA: struct (nullable = true)
 |    |-- APELLIDO: string (nullable = true)
 |    |-- CONTACTO: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- TIPO: string (nullable = true)
 |    |    |    |-- VALOR: string (nullable = true)
 |    |-- ID_PERSONA: long (nullable = true)
 |    |-- NOMBRE: string (nullable = true)



EMPRESA,FECHA,MONTO,PERSONA
"List(3, Apple)",2020-01-03,1383,"List(null, List(List(CORREO, owen@hotmail.com), List(CORREO, owen@gmail.com), List(TELEFONO, 991281120)), 18, Owen)"
"List(6, Google)",2020-01-04,2331,"List(null, null, 30, Clayton)"
"List(2, Microsoft)",2020-01-04,2280,"List(Parker, List(List(CORREO, vernon@hotmail.com), List(TELEFONO, 9912451237)), 47, Vernon)"
"List(1, Walmart)",2020-01-04,730,"List(null, List(List(CORREO, stephen@gmail.com), List(TELEFONO, 9910275611)), 28, Stephen)"
"List(4, Toyota)",2020-01-04,3081,"List(null, List(List(CORREO, erica@hotmail.com), List(TELEFONO, 912331240)), 91, Erica)"


In [ ]:
df2 = df.select(col('EMPRESA.ID_EMPRESA').alias('empresa_id'),
                col('EMPRESA.NOMBRE').alias('empresa_nombre'),
                col('PERSONA.ID_PERSONA').alias('persona_id'),
                col('PERSONA.NOMBRE').alias('persona_nombre'),
                col('PERSONA.APELLIDO').alias('persona_apellido'),
                explode(col('PERSONA.CONTACTO')).alias('explode'),
                col('FECHA').alias('fecha'),
                col('MONTO').alias('monto'))
df2.printSchema()
df2.display()

root
 |-- empresa_id: long (nullable = true)
 |-- empresa_nombre: string (nullable = true)
 |-- persona_id: long (nullable = true)
 |-- persona_nombre: string (nullable = true)
 |-- persona_apellido: string (nullable = true)
 |-- explode: struct (nullable = true)
 |    |-- TIPO: string (nullable = true)
 |    |-- VALOR: string (nullable = true)
 |-- fecha: string (nullable = true)
 |-- monto: long (nullable = true)



empresa_id,empresa_nombre,persona_id,persona_nombre,persona_apellido,explode,fecha,monto
3,Apple,18,Owen,null,"List(CORREO, owen@hotmail.com)",2020-01-03,1383
3,Apple,18,Owen,null,"List(CORREO, owen@gmail.com)",2020-01-03,1383
3,Apple,18,Owen,null,"List(TELEFONO, 991281120)",2020-01-03,1383
2,Microsoft,47,Vernon,Parker,"List(CORREO, vernon@hotmail.com)",2020-01-04,2280
2,Microsoft,47,Vernon,Parker,"List(TELEFONO, 9912451237)",2020-01-04,2280
1,Walmart,28,Stephen,null,"List(CORREO, stephen@gmail.com)",2020-01-04,730
1,Walmart,28,Stephen,null,"List(TELEFONO, 9910275611)",2020-01-04,730
4,Toyota,91,Erica,null,"List(CORREO, erica@hotmail.com)",2020-01-04,3081
4,Toyota,91,Erica,null,"List(TELEFONO, 912331240)",2020-01-04,3081


In [ ]:
df3 = df2.select(col('empresa_id'),
                 col('empresa_nombre'),
                 col('persona_nombre'),
                 col('persona_apellido'),
                 col('explode.TIPO').alias('persona_tipo_contacto'),
                 col('explode.VALOR').alias('persona_contacto'),
                 col('fecha'),
                 col('monto'))
df3.printSchema()
df3.display()

root
 |-- empresa_id: long (nullable = true)
 |-- empresa_nombre: string (nullable = true)
 |-- persona_nombre: string (nullable = true)
 |-- persona_apellido: string (nullable = true)
 |-- persona_tipo_contacto: string (nullable = true)
 |-- persona_contacto: string (nullable = true)
 |-- fecha: string (nullable = true)
 |-- monto: long (nullable = true)



empresa_id,empresa_nombre,persona_nombre,persona_apellido,persona_tipo_contacto,persona_contacto,fecha,monto
3,Apple,Owen,null,CORREO,owen@hotmail.com,2020-01-03,1383
3,Apple,Owen,null,CORREO,owen@gmail.com,2020-01-03,1383
3,Apple,Owen,null,TELEFONO,991281120,2020-01-03,1383
2,Microsoft,Vernon,Parker,CORREO,vernon@hotmail.com,2020-01-04,2280
2,Microsoft,Vernon,Parker,TELEFONO,9912451237,2020-01-04,2280
1,Walmart,Stephen,null,CORREO,stephen@gmail.com,2020-01-04,730
1,Walmart,Stephen,null,TELEFONO,9910275611,2020-01-04,730
4,Toyota,Erica,null,CORREO,erica@hotmail.com,2020-01-04,3081
4,Toyota,Erica,null,TELEFONO,912331240,2020-01-04,3081


Si quisieramos trabajar con elementos especificos de un 'array', en este caso del array CONTACTO.

In [ ]:
df4 = df.select(col('PERSONA.NOMBRE'),
                col('PERSONA.CONTACTO').getItem(0).alias('TIPO'),
                col('PERSONA.CONTACTO').getItem(0).alias('CONTACTO'))

df4.printSchema()
df4.display()

root
 |-- NOMBRE: string (nullable = true)
 |-- TIPO: struct (nullable = true)
 |    |-- TIPO: string (nullable = true)
 |    |-- VALOR: string (nullable = true)
 |-- CONTACTO: struct (nullable = true)
 |    |-- TIPO: string (nullable = true)
 |    |-- VALOR: string (nullable = true)



NOMBRE,TIPO,CONTACTO
Owen,"List(CORREO, owen@hotmail.com)","List(CORREO, owen@hotmail.com)"
Clayton,null,null
Vernon,"List(CORREO, vernon@hotmail.com)","List(CORREO, vernon@hotmail.com)"
Stephen,"List(CORREO, stephen@gmail.com)","List(CORREO, stephen@gmail.com)"
Erica,"List(CORREO, erica@hotmail.com)","List(CORREO, erica@hotmail.com)"


In [ ]:
df5 = df.select(col('PERSONA.NOMBRE'),
                col('PERSONA.CONTACTO').getItem(0)['TIPO'].alias('TIPO_1'),
                col('PERSONA.CONTACTO').getItem(0)['VALOR'].alias('CONTACTO_1'))

df5.printSchema()
df5.display()

root
 |-- NOMBRE: string (nullable = true)
 |-- TIPO_1: string (nullable = true)
 |-- CONTACTO_1: string (nullable = true)



NOMBRE,TIPO_1,CONTACTO_1
Owen,CORREO,owen@hotmail.com
Clayton,null,null
Vernon,CORREO,vernon@hotmail.com
Stephen,CORREO,stephen@gmail.com
Erica,CORREO,erica@hotmail.com


##### Ejemplo 7

In [ ]:
from pyspark.sql.functions import *
df = spark.read.option('inferschema','True').json('/FileStore/tables/json/customer_obj.json')
df.printSchema()
df.show(truncate=False)

root
 |-- courses: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- customerid: long (nullable = true)
 |-- customername: string (nullable = true)
 |-- details: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- mobile: string (nullable = true)
 |-- registered: boolean (nullable = true)

+------------------------+----------+------------+-----------------+----------+
|courses                 |customerid|customername|details          |registered|
+------------------------+----------+------------+-----------------+----------+
|[AZ-900, AZ-500, AZ-303]|1         |UserA       |{CityA, 111-1112}|true      |
|[AZ-104, AZ-500, DP-200]|2         |UserB       |{CityB, 333-1112}|true      |
+------------------------+----------+------------+-----------------+----------+



In [ ]:
newdf = df.select(col("customerid"),col("customername"),col("registered"),explode(col("courses")),col("details.city"),col("details.mobile"))
newdf.show(truncate=False)

+----------+------------+----------+------+-----+--------+
|customerid|customername|registered|col   |city |mobile  |
+----------+------------+----------+------+-----+--------+
|1         |UserA       |true      |AZ-900|CityA|111-1112|
|1         |UserA       |true      |AZ-500|CityA|111-1112|
|1         |UserA       |true      |AZ-303|CityA|111-1112|
|2         |UserB       |true      |AZ-104|CityB|333-1112|
|2         |UserB       |true      |AZ-500|CityB|333-1112|
|2         |UserB       |true      |DP-200|CityB|333-1112|
+----------+------------+----------+------+-----+--------+



#### Escribir un archivo JSON

##### Forma 1

In [ ]:
json_df.coalesce(1).write.json('/FileStore/tables/json/', mode='overwrite')

##### Forma 2

In [ ]:
json_df.coalesce(1).write.format('json').save('/FileStore/tables/json/', mode='overwrite')

##### Forma 3

In [ ]:
json_df.write.json('/FileStore/tables/json/', mode='overwrite')

##### Forma 4

In [ ]:
json_df.coalesce(1).write.format('json').save('/FileStore/tables/json/', mode='overwrite', compression='gzip')